In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# import time

In [9]:
url = "https://remoteok.com/remote-engineer-jobs"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}


response = requests.get(url, headers=headers)

if response.status_code == 200:
    print("Page fetched successfully!")
else:
    print("An error occurred while fetching the page")

Page fetched successfully!


In [10]:
# Parse the HTML content
soup = BeautifulSoup(response.text, "html.parser")

# Print the page title to confirm it worked
print(soup.title.text)
# print(soup.prettify())

Remote Engineer Jobs with great benefits and pay


In [27]:
# Find all job elements
jobs = soup.find_all("tr", class_="job")


# Initialize empty lists to store the data
job_titles = []
companies = []
locations = []
salaries = []
jobTags = {}

csv_data = []

# Loop through each job element
for job in jobs:
    # extract job titles
    title = job.find("td", class_="company_and_position").find("h2").text.strip()
    job_titles.append(title)

    company = job.find("td", class_="company_and_position").find("h3").text.strip()
    companies.append(company)

    location = (
        job.find("td", class_="company_and_position")
        .find("div", class_="location")
        .text.strip()
    )
    locations.append(location)

    salary = (
        job.find("td", class_="company_and_position")
        .find("div", class_="location")
        .find_next_sibling("div", class_="location tooltip")
        .text.strip()
    )
    salaries.append(salary)

    tags = job.find("td", class_="tags").find_all("h3")
    # put all tags in a list
    tagList = []
    for tag in tags:
        tagList.append(tag.text.strip())
    jobTags[title] = list(set(tagList))

    csv_data.append(
        {
            "job_title": title,
            "company": company,
            "location": location,
            "salary": salary,
            "tags": ",".join(list(set(tagList))),
        }
    )

# make a dataframe
df = pd.DataFrame(csv_data)

# save the data to a csv file
df.to_csv("remoteok_jobs.csv", index=False, encoding="utf-8")